In [ ]:
import pandas as pd
import numpy as np

import math
import numpy as np, matplotlib.pyplot as plt
from scipy.integrate import odeint
import sys

try:
    from tqdm.notebook import tqdm
except Exception:
    print('For a nice progress bar install tqdm (pip3 install tqdm)', file=sys.stderr)
    def tqdm(iterable, *args, **kwargs):
        return iterable

    
def progress_wrapper(iterable, level, max_progress_level):
    if level == 0:
        return tqdm(iterable)
    elif level <= max_progress_level:  # configurable
        return tqdm(iterable, leave=False)
    else:
        return iterable
    

def compute_loss(time_points, predicted_metrics, true_metrics):
    loss_sum = 0.0
    # Iterace through infected, recovered and dead
    for predicted, true in zip(predicted_metrics, true_metrics):
        true = true.to_numpy().astype('float64')
        assert predicted.shape[0] == true.shape[0], "expected solution's size to be {}, but got {}".format(true.shape[0], predicted.shape[0])
#         max_values = np.stack([true, predicted, np.full_like(true, 0.0001)], axis=1).max(axis=1)
        loss_sum += np.mean((true - predicted) ** 2)
    return loss_sum


def cut_metrics(metrics, from_idx, to_idx):
    if to_idx is None:
        return [x[from_idx:] for x in metrics]
    else:
        return [x[from_idx:to_idx] for x in metrics]


def fit(ode_system_getter, time_points, params, to_metrics, true_metrics, max_progress_level=1, val_samples=0):
    """
    ode_system_getter: функция, которая получает набор параметров из params и возвращает tuple,
                       в котором первый элемент – функция, задающая систему уравнений
                       (и принимающая только параметры y и t)
                       и соответствующая аргументу func функции scipy.integrate.odeint,
                       а второй элемент – вектор начальных условий y0 функции scipy.integrate.odeint
    time_points: точки, для которых нужно решить систему уранений. Аналогично параметру t в scipy.integrate.odeint
    params: список, в котором каждый i-тый элемент –
            список возможных значений i-того параметра, передаваемого в ode_system_getter
    to_metrics: функция, принимающая solution (результат функции scipy.integrate.odeint)
                и отдающая список метрик, по которым будет производиться оценка параметров.
                каждая метрика – одномерный numpy-массив значений
    true_metrics: список метрик для данных, под которые мы подстраиваемся
                  каждая метрика – одномерный numpy-массив значений
    max_progress_level: глубина вложенных циклов, для которых будет отображаться progress-bar. Считается от 0.
    val_samples: количество элементов, отсекаемых на валидацию.
                 целое число означает количество элементов,
                 дробное в интервале (0;1) – долю от общего числа элементов
                  
    
    """
    
    val_len = math.ceil(len(time_points)*val_samples) if val_samples < 1 else val_samples
    train_len = len(time_points) - val_len
    assert train_len > 0, 'Too big value for val_samples, no train samples left'
    if val_len <= 0:
        print('Too small value for val_samples, no validation loss will be computed', file=sys.stderr)
    train_points = time_points[:train_len]
    val_points = time_points[train_len:]
    true_train = cut_metrics(true_metrics, 0, train_len)
    true_val = cut_metrics(true_metrics, train_len, None)
    
    def fit_stage(left_params, current_values, level):
        if left_params:
            best_train_loss = None
            best_val_loss = None
            best_solution = None
            best_params = None
            for param_value in progress_wrapper(left_params[-1], level, max_progress_level):
                new_values = current_values + (param_value,)
                return_value = fit_stage(left_params[:-1], new_values, level+1)
                if return_value is None:
                    continue
                train_loss, val_loss, solution, params_for_loss = return_value
                if best_train_loss is None or best_train_loss > train_loss:
                    best_train_loss = train_loss
                    best_val_loss = val_loss
                    best_solution = solution
                    best_params = params_for_loss
            return best_train_loss, best_val_loss, best_solution, best_params
        else:
            return_value = ode_system_getter(*list(reversed(current_values)))
            if return_value is None:  # e.g. invalid parameters
                return None
            ode_system, initial_conditions = return_value
            solution = odeint(ode_system, initial_conditions, time_points)
            train_loss = compute_loss(train_points, cut_metrics(to_metrics(solution), 0, train_len), true_train)
            if val_len > 0:
                val_loss = compute_loss(val_points, cut_metrics(to_metrics(solution), train_len, None), true_val)
            else:
                val_loss = 0.0
            return train_loss, val_loss, solution, current_values
    
    best_train_loss, val_loss, best_solution, best_params = fit_stage(params, tuple(), 0)
    return best_train_loss, val_loss, best_solution, tuple(reversed(best_params))


# Небольшая функция для быстрой отрисовки графичков, объединяющая реальные данные и смоделированные
def plot_data_together(time_points, predicted_data, true_data, upper_limit, labels=None, log_lower_bound=1e-3):
    colors = ['b', 'g', 'r', 'y', 'k', 'm']
    if not labels:
        labels = ['']*len(true_data)
    
    plt.figure(figsize=(13,5))
    plt.subplot(1,2,1)
    for y_pred, y_true, color, label in zip(predicted_data, true_data, colors, labels):
        plt.plot(time_points, y_true, color+'--', label=label+' (true)', alpha=0.7)
        plt.plot(time_points, y_pred, color, label=label+' (pred)')
        
    plt.xlabel("Time (days)")
    plt.ylabel("Number of people")
    plt.legend()
    plt.ylim([0, upper_limit])

    #Same plot but on log scale
    plt.subplot(1,2,2)
    for y_pred, y_true, color, label in zip(predicted_data, true_data, colors, labels):
        plt.plot(time_points, y_true, color+'--', label=label+' (true)', alpha=0.7)
        plt.plot(time_points, y_pred, color, label=label+' (pred)')
    plt.semilogy()
    plt.xlabel("Time (days)")
    plt.ylabel("Number of people")
    plt.legend()
    plt.ylim([log_lower_bound, upper_limit])

# SEIR Absolute

In [ ]:
data_sweden_confirmed = pd.read_csv('datasets_583052_1271240_time_series_confimed-confirmed.csv')
infected = data_sweden_confirmed[[i for i in data_sweden_confirmed.columns if '2020-' in i]].iloc[-1].fillna(0)

In [ ]:
data_sweden_deaths = pd.read_csv('datasets_583052_1271240_time_series_deaths-deaths.csv')
deaths = data_sweden_deaths[[i for i in data_sweden_deaths.columns if '2020-' in i]].iloc[-1].fillna(0)

In [ ]:
data_sweden_recovery = pd.read_csv('time_series_covid19_recovered_global.csv')
recovered = data_sweden_recovery[[i for i in data_sweden_recovery.columns if '/20' in i]].iloc[203][3:-10].fillna(0)

In [ ]:
recovered = '0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,16,16,16,16,16,16,16,16,16,16,16,16,16,103,103,205,205,205,205,205,205,205,381,381,381,381,381,381,550,550,550,550,550,550,550,550,1005,1005,1005,1005,1005,1005,1005,1005,1005,1005,4074,4074,4074,4971,4971,4971,4971,4971,4971,4971,4971,4971,4971,4971,4971,4971,4971,4971,4971,4971,4971,4971,4971,4971,4971,4971,4971,4971,4971'
recovered = pd.DataFrame([int(i) for i in recovered.split(',')[3:-16]])

In [ ]:
import math
from itertools import accumulate

ITALY_POPULATION = 60461826
SPAIN_POPULATION = 45619980
SWEDEN_POPULATION = 10099265
N=ITALY_POPULATION

data = pd.read_csv('italy.csv')
start_day = 40
data = data[start_day:].reset_index()
data['Выздоровлений'] += data['Смертей']
data['Заражений'] -= data['Выздоровлений']
# data = {
#     'Заражений' : np.array(list(accumulate(map(int, infected[start_day:]))))[:100],
#     'Выздоровлений' : np.array(list(accumulate(map(int, list(recovered[start_day:]) + list(deaths[start_day:])))))[:100]
# }
true_ird = (
    pd.DataFrame(data['Заражений']),
    pd.DataFrame(data['Выздоровлений'])
)
time_points = np.arange(true_ird[0].shape[0])

def func_getter(r1, r2, b1, b2, alpha, gamma):
    # Если набор параметров невалиден, можно просто вернуть None, тогда такой набор будет пропущен
    if r1 == r2 == 0:
        return
    if b1 == b2 == 0:
        return
    if alpha == 0:
        return
    if gamma == 0:
        return
    
    initial_conditions = np.zeros(4)
    initial_conditions[0] = 0
    initial_conditions[1] = data['Заражений'][8] - data['Заражений'][0]
    initial_conditions[2] = data['Заражений'][0]
    initial_conditions[3] = data['Выздоровлений'][0]
    initial_conditions[0] = N - np.sum(initial_conditions)
    
    def ode_system(y,t): 
        # Функция с системой НЕ ПРИНИМАЕТ дополнительные параметры,
        # Но они автоматически "подсасываются" из-за вложенности в func_getter
        S, E, I, R = y
        
        dS = - (r1 * b1 * I * S) / N - (r2 * b2 * E * S) / N
        dE = (r1 * b1 * I * S) / N + (r2 * b2 * E * S) / N - alpha * E
        dI = alpha * E - gamma * I
        dR = gamma * I
        
        dy = [dS, dE, dI, dR]
        
        return dy
    
    return ode_system, initial_conditions

# Наши метрики: infected, recovered, dead
def solution_to_IRD(solution):
    infected = solution[:, 2]
    recovered = solution[:, 3]
    return infected, recovered

In [ ]:
params = [
    np.linspace(0, 40, 5),
    np.linspace(0, 15, 5),
#     [0],
    np.linspace(0, 0.5, 5),
    np.linspace(0, 1, 5),
#     [0],
    np.linspace(0, 1, 5),
    np.linspace(0, 1, 5),
]

train_loss, val_loss, best_solution, best_params = fit(
    func_getter,
    time_points,
    params,
    solution_to_IRD,
    true_ird,
    val_samples=10,
    max_progress_level=1,
)
print(round(train_loss*1000, 4), round(val_loss*1000, 4), best_params)


In [ ]:
plot_data_together(
    time_points, solution_to_IRD(best_solution), true_ird, N/50,
#     labels=["Infected", "Recovered", "Dead"],
    labels=["Infected", "Recovered"],
    log_lower_bound=1
)

In [ ]:
italy_best = [0, 11.0, 0, 0.05555555555555556, 0.6222222222222222, 0.02777777777777778]

In [ ]:
spain_best = [0, 7.0, 0, 0.02631578947368421, 0.22105263157894733, 0.04052631578947369]